In [15]:
import requests
from massive import RESTClient
from datetime import datetime
from datetime import timedelta

In [16]:
class Analyze:
    def __init__(self, client):
        self.client = client
        self.today = datetime.today().strftime('%Y-%m-%d')

    """
        Converts milliseconds timestamp to more readable format
    """
    def convert_timestamp(self, ts):
        return datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d')

    """
        Gets prices for a specific ticker over a set time
    """
    def get_price(self, ticker, multiplier=1, timespan='day', from_date=None):
        self.time_dict = {}
        for _ in self.client.list_aggs(
                ticker=ticker, 
                multiplier=multiplier, 
                timespan=timespan, 
                from_=from_date, 
                to=self.today
        ):
            self.time_dict[self.convert_timestamp(_.timestamp)] = {
                'open': _.open, 
                'close': _.close, 
                'volume': _.volume, 
                'high': _.high, 
                'low': _.low,
                'transactions': _.transactions
            }

    """
        Gets RSI values over time frame and for a specific ticker
    """
    def get_rsi(
            self, 
            ticker, 
            timespan='day',
            adjusted=True, 
            window=14, 
            series_type='close',    
            order='desc', 
            limit=10
    ):
        rsi_data = self.client.get_rsi(
            ticker=ticker,
            timespan=timespan,
            adjusted=adjusted,
            window=window,
            series_type=series_type,
            order=order,
            limit=limit
        )
        for _ in rsi_data.values:
            self.time_dict[self.convert_timestamp(_.timestamp)]['rsi'] = _.value

In [17]:
"""
    use api key and set up client
"""
api_key = 'hJQGYwLtVkOmM45yOBeEOg99YqOzJAz9'
client = RESTClient(api_key=api_key)

"""
    set day range to use (will be adjusted for my time frames)
"""
day_range = 21
from_date = datetime.today() - timedelta(days=day_range + 1)
from_date = from_date.strftime('%Y-%m-%d')

ticker = 'AAPL'

"""
    instance of class
"""
inst = Analyze(client=client)

inst.get_price(
    ticker=ticker,
    multiplier=1,
    from_date=from_date
)

inst.get_rsi(
    ticker=ticker,
    timespan='day',
    adjusted=True,
    window=14,
    series_type='close',
    order='desc',
    limit=day_range
)

MaxRetryError: HTTPSConnectionPool(host='api.massive.com', port=443): Max retries exceeded with url: /v2/aggs/ticker/AAPL/range/1/day/2025-10-16/2025-11-07 (Caused by ResponseError('too many 429 error responses'))